In [15]:
import numpy as np
import brian2 as b2
import matplotlib.pyplot as plt

Notebook goal is to model gridcells as a twisted torus using velocity as an input, using equations from Guanella and Verschure 2007

**Reference**
\
\
$N:$ Number of neurons
\
$N_{x}:$ Width
\
$N_{y}:$ Height
\
$A:$ Activity matrix of shape $x$ by $y$
\
$I:$ Intensity parameter, defining the overall strength of the synapses
\
$\sigma:$ Regulates the size of the Gaussian
\
$T:$ Shift parameter determining excitatory and inhibitory zones
\
$\alpha:$ The gain (distance between receptive fields)
\
$\beta:$ The bias (angle of receptive fields) $[0, \pi/3]$

In [16]:
#Only Ny needs to be defined, but writing out for clarity
#Nx = Ny + 1, N = Nx * Ny
Ny = 9
Nx = 10
N = 90

Initialize neuron activity randomly distributed between $0$ and $1/\sqrt{N}$

In [17]:
def initA(Nx, Ny):
    return np.random.uniform(0, 1/np.sqrt(N), N)

**Activity Eqs.**
\
\
$B_{i}(t+1) = A_{i}(t) + \sum \limits _{j=1}^{N}A_{j}(t)w_{ji}$ 
\
$A_{i}(t+1) = B_{i}(t+1) + \tau\Bigg(\frac{B_{i}(t+1)}{<B_{j}(t+1)>_{j=1}^{N}}-B_{i}(t+1)\Bigg)$
\
\
Function $< . >_{j=1}^{N}$ is the mean over the cells of the network, used to ensure stability \
\
Therefore $<B_{j}(t+1)>_{j=1}^{N}$ is an external cell that keeps things stable \
\
$A_{i}(t+1) = 0$ when $A_{i}(t+1)$ is smaller than 0 \
\
\
**Synapse Eqs.**
\
\
$w_{ij} = I\exp{(-\frac{||c_{i}-c_{j}||_{tri}^2}{\sigma^2})} - T$
\
\
Where $c_{i} = (c_{ix},c_{iy})$ is the position of cell $i$, $c_{ix} = (i_{x} - 0.5)/N_{x}$ and $c_{iy} = \frac{\sqrt{3}}{2}(i_{y} - 0.5)/N_{y}$
\
\
$dist_{tri}(c_{i},c_{j}) := ||c_{1}-c_{2}||_{tri} = \min_{j=1}^{7}||c_{1}-c_{2}+s_{j}||$




where:
\
\
$s_{1} := (0,0),$
\
\
$s_{2} := (-0.5,\frac{\sqrt{3}}{2}),$
\
\
$s_{3} := (-0.5,-\frac{\sqrt{3}}{2}),$
\
\
$s_{4} := (0.5,\frac{\sqrt{3}}{2}),$
\
\
$s_{5} := (0.5,-\frac{\sqrt{3}}{2})$
\
\
$s_{6} := (-1,0),$
\
\
$s_{7} := (1,0),$

**Modulation Eqs**
\
\
$R_{\beta} = \begin{bmatrix} \cos(\beta) & -\sin(\beta) \\ \sin(\beta) & \cos(\beta) \end{bmatrix}$

$w_{ij}(t+1) = I\exp\bigg(-\frac{||c_{i}-c_{j}+\alpha R_{\beta}v(t)||^2_{tri}}{\sigma^2}\bigg) - T$

To calculate activity $A$, we need the weight matrix $W$. To calculate the weights, we need the receptive field locations $(c_{x},c_{y})$, which are stored in the matrix $C$

In [5]:
C = np.zeros((90,2))

$i_{x}, i_{y}$ are indices in the range $0$ to $N_{x}, N_{y}$, whilst $i$ is the index in range $0$ to $N$ 

In [6]:
i=0
yConst = np.sqrt(3)/2
for ix in range(Nx):
    for iy in range(Ny):
        C[i] = [(ix - 0.5)/Nx, yConst*(iy - 0.5)/Ny]
        i+=1

Now we can use this to calculate the distribution

In [14]:
sArray = np.asarray([[0,0],[-0.5, yConst],[-0.5, -yConst],[0.5,yConst],[0.5,-yConst],[-1,0],[1,0]])

def minFunc(ci, cj, sArray):
    """
    :param array ci: Receptive field coords of cell i.
    :param array cj: Receptive field coords of cell j.
    :param array sArray: List of s values.
    """
    storage = np.zeros_like(sArray)
    
    for i in range(sArray):
        storage[i] = np.abs(ci - cj + sArray[i])

    return np.min(storage)

In [25]:
#Total time
T = 1000
#Activity trace
At = np.zeros((T,Ny,Nx))
#Initialize Activity
At[0] = initA(Nx, Ny)

# t = 0
# for A in At:
#     u = 0
#     t+=1
#     for y in A:
#         v = 0
#         u+=1
#         for x in y:
#             v+=1
#             Bt = At[t][u][v] + 

In [28]:
At[0]

array([[0.09097449, 0.00213466, 0.04824029, 0.10033655, 0.04174819,
        0.01321521, 0.03707262, 0.04270762, 0.03799142, 0.09872167],
       [0.10523155, 0.0851333 , 0.04658369, 0.07059163, 0.01177187,
        0.03113697, 0.04381726, 0.07437927, 0.03485283, 0.04474682],
       [0.07182237, 0.01866613, 0.07848858, 0.01107906, 0.00989162,
        0.08411841, 0.00588713, 0.10120159, 0.0377117 , 0.0085701 ],
       [0.02772478, 0.10412534, 0.01035195, 0.00861389, 0.09962828,
        0.04940636, 0.04877016, 0.00579517, 0.04611251, 0.00622513],
       [0.10174544, 0.10384373, 0.00388209, 0.08369128, 0.04063739,
        0.00819655, 0.00685105, 0.05490134, 0.07825399, 0.07833923],
       [0.0567969 , 0.06036354, 0.04520778, 0.0897016 , 0.0846288 ,
        0.08046624, 0.02555999, 0.0041462 , 0.05142996, 0.09541249],
       [0.01195905, 0.07071124, 0.07412817, 0.09110194, 0.04355289,
        0.04124021, 0.07120034, 0.05794961, 0.08444353, 0.10519736],
       [0.09791748, 0.01815966, 0.0611371

In [29]:
At[0][0]

array([0.09097449, 0.00213466, 0.04824029, 0.10033655, 0.04174819,
       0.01321521, 0.03707262, 0.04270762, 0.03799142, 0.09872167])